In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Activation, Dropout, BatchNormalization
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [ ]:
### Functional model
# inputs = Input(shape=(4,))
# x = Dense(6, activation='relu')(inputs)
# x = Dense(6, activation='relu')(x)
# outputs = Dense(3, activation='softmax')(x)

In [ ]:
#Sequential model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape = (4,)))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization(momentum=0.8))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=1000, verbose=0)

In [ ]:
model.history.history.get('acc')[-1]

In [ ]:
plt.plot(model.history.history.get('acc'),'g')

In [ ]:
plt.plot(model.history.history.get('loss'),'r')

In [ ]:
pred = model.predict(X_test)
pred2 = np.argmax(pred, axis=1)

In [ ]:
accuracy_score(pred2, y_test)

# Running it with Grid Search

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def build_classifier(optimizer):
    model = Sequential()
    
    model.add(Dense(16, activation='relu', input_shape = (4,)))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size':[1,5],
              'epochs':[100,200], 
              'optimizer':['adam', 'rmsprop']}
parameters

In [ ]:
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=3)

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
pred = grid_search.predict(X_test)
#pred2 = np.argmax(pred, axis=1)

In [ ]:
accuracy_score(pred, y_test)

# LSTM

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('job_skills.csv')

In [ ]:
df.shape

In [ ]:
df1 = df.drop(['Company', 'Location', 'Category'], axis=1)

In [ ]:
y = df['Category']

In [ ]:
def concatenate(row):
    return ' '.join([str(i) for i in row])

In [ ]:
df1['combined'] = [concatenate(list(df1.iloc[i])) for i in range(len(df1))]

In [ ]:
X = df1['combined']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)

In [ ]:
[len(i) for i in [X_train, X_test, y_train, y_test]]

In [ ]:
cnt = CountVectorizer(stop_words='english')

In [ ]:
features = cnt.fit_transform(X_train)

In [ ]:
features.shape

In [ ]:
lbl = LabelEncoder()
encoded_labels = lbl.fit_transform(y_train)

In [ ]:
len(np.unique(encoded_labels))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

In [ ]:
max_features = features.shape[1]
max_features

In [ ]:
model = Sequential()
model.add(Embedding(max_features, output_dim=128))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(23, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(features, 
          encoded_labels, 
          epochs=1, 
          batch_size=16)

In [ ]:
plt.plot(model.history.history.get('acc'))